In [3]:
import sys
sys.path.append('..')

In [4]:
from models.gate.gate import *
from models.gate.processor import *
from models.gate.trainer import *

2024-11-13 21:27:37.561286: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 21:27:37.641135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731544057.676025  146939 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731544057.686559  146939 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 21:27:37.765033: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
data_path = '/kaggle/input/cic-ids-2018-preprocessing/all_data.parquet'

# Initialize DataProcessor
processor = DataProcessor(data_path)
processor.load_and_preprocess_data()
processor.encode_labels()
X_train, X_test, y_train, y_test = processor.split_data()
class_weight_dict = processor.compute_class_weights(y_train)

# Create a MirroredStrategy for using multiple GPUs
strategy = tf.distribute.MirroredStrategy()

# Build and train models
model_builder = ModelBuilder(
    input_shape=X_train.shape[1],
    num_classes=processor.num_classes,
    strategy=strategy
)

model_build_methods = {
    'mlp_residual': model_builder.build_mlp_residual_model,
    'cnn': model_builder.build_cnn_model,
    'lstm': model_builder.build_lstm_model,
    'attention': model_builder.build_attention_model,
}

for model_name, build_method in model_build_methods.items():
    print(f'\nTraining {model_name} model')
    if model_name == 'attention':
        model = build_method(num_heads=4)  # Adjust num_heads as needed
    else:
        model = build_method()
    trainer = ModelTrainer(model=model)
    trainer.compile_model(strategy, learning_rate=1e-3)
    trainer.train_model_per_epoch(X_train, y_train, X_test, y_test, class_weight_dict, epochs=100, batch_size=24576)
    trainer.evaluate_model(X_test, y_test)
    trainer.save_history(model_name, '../out/gate')
    trainer.save_classification_report(model_name, '../out/gate')
    trainer.save_model(model_name, '../out/gate')
    trainer.clean_up()

# Clean up data to free memory
del X_train, X_test, y_train, y_test
gc.collect()